# Vacan text preprocessing

In [68]:
import pandas as pd
import bs4
import re
from IPython.display import display

In [4]:
vacancies = pd.read_pickle('data/vacancies.pickle')

In [7]:
vacancies.head()

,id,name,html,url,site,date
3,1413553876,Разработчик Ruby/Ruby on Rails,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11833695?query=%D...,hh.ru,2014-10-17 13:51:12
4,1413553877,Ведущий программист 1 С/Руководитель направлен...,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11926858?query=%D...,hh.ru,2014-10-17 13:51:12


In [19]:
vacancies.date.max()

Timestamp('2015-12-06 12:39:17')

In [15]:
soup = bs4.BeautifulSoup(vacancies.iloc[0].html, 'html.parser')

In [20]:
soup.meta

<meta content="RUR" itemprop="salaryCurrency"><meta content="70000" itemprop="baseSalary">от 70 000 руб.</meta></meta>

In [108]:
def get_salary(html: str) -> tuple:
    soup = bs4.BeautifulSoup(html, 'html.parser')
    basesalary = None
    salarycurrency = None
    for meta in soup.findAll("meta"):
        meta_name = meta.get('itemprop', '').lower()
        if meta_name == 'salarycurrency':
            salarycurrency = meta.get('content', '').lower()
        if meta_name == 'basesalary':
            basesalary = int(meta.get('content', ''))
    return salarycurrency, basesalary

def get_text(html: str) -> str:
    try:
        x = re.search(r'</table>.*</div>(.*)Тип занятости', html)
        return x.groups()[0].lower()
    except AttributeError:
        return html.lower()

In [109]:
example_html = vacancies.iloc[5].html
display(get_salary(example_html))
display(get_text(example_html))

('rur', 100000)

'компания «эспокада» (один из лидеров рынка домашнего текстиля) предлагает вам поучаствовать в конкурсе на замещение вакансии «программист-разработчик 1с».   что мы можем предложить вам:  работа в стабильной успешной компании просторный офис в 5 минутах ходьбы от метро «авиамоторная» комфортная рабочая обстановка молодой и дружный коллектив сложные, интересные и амбициозные задачи простор для реализации своего интеллектуального и творческого потенциала перспективы карьерного роста оформление в соответствии с тк рф    необходимые требования:  высшее образование (техническое) релевантный опыт работы от 6 лет понимание бизнес-процессов предприятий оптовой торговли (закупки, склад, производство, транспорт, продажи, клиентский сервис, бухгалтерия) способность мыслить нестандартно умение планировать свое время и всегда доводить работу до конца высокий уровень самоорганизации и ответственности за свою работу знания продуктов семейства 1с и практические навыки работы с ними отличное знание вст

In [110]:
vacancies['text'] = vacancies.html.apply(get_text)

In [91]:
salaries = vacancies.html.apply(get_salary)

In [111]:
vacancies['salary'] = salaries.apply(lambda x: x[1])

In [112]:
vacancies['salary_currency'] = salaries.apply(lambda x: x[0])

In [113]:
vacancies['name'] = vacancies.name.apply(lambda x: x.lower())

In [114]:
vacancies.head(2)

,id,name,html,url,site,date,salary,salary_currency,text
3,1413553876,разработчик ruby/ruby on rails,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11833695?query=%D...,hh.ru,2014-10-17 13:51:12,70000,rur,"в состав распределенной команды, разрабатывающ..."
4,1413553877,ведущий программист 1 с/руководитель направлен...,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11926858?query=%D...,hh.ru,2014-10-17 13:51:12,110000,rur,медицинская сеть клиник приглашает на работу н...


In [117]:
vacancies_prepared = vacancies[['date', 'name', 'text', 'salary', 'salary_currency']]

In [122]:
vacancies_prepared['currency'] = vacancies_prepared.loc[:,'salary_currency']

/usr/local/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [123]:
del vacancies_prepared['salary_currency']

In [124]:
vacancies_prepared.head()

,date,name,text,salary,currency
3,2014-10-17 13:51:12,разработчик ruby/ruby on rails,"в состав распределенной команды, разрабатывающ...",70000,rur
4,2014-10-17 13:51:12,ведущий программист 1 с/руководитель направлен...,медицинская сеть клиник приглашает на работу н...,110000,rur
5,2014-10-17 13:51:12,java-разработчик,обязанности: участие в проектах компании по р...,40000,rur
8,2014-10-17 13:51:12,ios разработчик,"привет. меня зовут егор, в 2011 году мы создал...",120000,rur
9,2014-10-17 13:51:12,программист php (web-разработка; back-end),в телекоммуникационную компанию требуется прог...,80000,rur


In [125]:
vacancies_prepared.to_pickle('data/vacancies_prepared.pickle')